In [ ]:
import openai
import csv
import os
from dotenv import load_dotenv
import requests
import pandas as pd


In [ ]:
def read_questions(file_path):
    with open(file_path, 'r') as file:
        questions = file.readlines()
    return [q.strip() for q in questions]

In [ ]:
def get_gpt4_response(question):
    response = openai.ChatCompletion.create(
      model="gpt-4-0125-preview",  # cheapest and newest model right now
      messages=[{"role": "user", "content": question}]
    )
    print(response.choices[0].message['content'].strip())
    return response.choices[0].message['content'].strip()

In [ ]:
def api_response(text, api_key):
    url = "https://api.thebipartisanpress.com/api/endpoints/beta/robert/"
    setup = {"API": api_key, "Text": text}
    response = requests.post(url, data=setup)
    print(response.text)
    return response.text


In [ ]:
def write_response_to_csv(question, response, score, csv_file):
    # Open the file in append mode ('a') to add each row without overwriting
    with open(csv_file, 'a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow([question, response, score])

In [ ]:
def main():
    load_dotenv()
    openai.api_key = os.getenv('OPENAI_API_KEY')
    bias_api_key = os.getenv('BIAS_API_KEY')

    questions = read_questions('questions.txt')
    csv_file = 'gpt-4-0125-preview_responses.csv'
    
    # Initialize the CSV file with headers before processing questions
    with open(csv_file, 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['Question', 'Response', 'Score'])
    
    for question in questions:
        response = get_gpt4_response(question)
        score = api_response(response, bias_api_key)
        write_response_to_csv(question, response, score, csv_file)

In [ ]:
main()